In [26]:
import pyvista as pv
import numpy as np
from stl import mesh

import matplotlib.pyplot as plt
import time

In [3]:
fp = "../../real_data/02_GrotteDeLaCascadeDeMotiers/output/3rdGallery_cropped_not_filled_25cm.stl"


In [27]:
my_mesh = mesh.Mesh.from_file(fp)

In [30]:
nb_search = np.loadtxt("Bourbier_nb_search.txt").astype(int)

In [31]:
nb_search

array([[    1, 22927,  7111],
       [    2, 22932,     0],
       [    3,     4,     1],
       ...,
       [26076, 29454, 29474],
       [10420, 29468, 29473],
       [29467, 26078, 29472]])

In [33]:
print(my_mesh.vectors[1])


[[823.849   177.78467 764.1065 ]
 [824.099   177.58751 763.99005]
 [823.849   177.58751 764.0701 ]]


In [4]:
pv_mesh = pv.read(fp)

In [10]:
pv_mesh_surface = pv_mesh.extract_surface().triangulate()
assert pv_mesh_surface.is_all_triangles

In [16]:
pv_mesh_surface.faces

array([    3,     0,     1, ..., 14803, 14075, 14078], dtype=int64)

In [17]:
faces = pv_mesh_surface.faces.reshape((-1,4))[:, 1:4]


In [23]:
# https://github.com/pyvista/pyvista-support/issues/96

def find_faces_with_node(index):
    """Pass the index of the node in question.
    Returns the face indices of the faces with that node."""
    return [i for i, face in enumerate(faces) if index in face]

def find_neighbor_faces(index):
    """Pass the face index. 
    Returns the indices of all neighboring faces"""
    face = faces[index]
    sharing = set()
    for vid in face:
        [sharing.add(f) for f in find_faces_with_node(vid)]
    sharing.remove(index)
    return list(sharing)
    

def find_neighbor_faces_by_edge(index):
    """Pass the face index. 
    Returns the indices of all neighboring faces with shared edges."""
    face = faces[index]
    a = set(f for f in find_faces_with_node(face[0]))
    a.remove(index)
    b = set(f for f in find_faces_with_node(face[1]))
    b.remove(index)
    c = set(f for f in find_faces_with_node(face[2]))
    c.remove(index)
    return [list(a.intersection(b))[0], 
            list(b.intersection(c))[0], 
            list(a.intersection(c))[0]]

In [32]:
n = faces.shape[0]

time_in = time.time()

for idx in range(50):
    print(find_neighbor_faces_by_edge(idx))
    
time_out = time.time()

print((time_out - time_in) / 50)


[1, 22923, 7111]
[2, 22928, 0]
[3, 4, 1]
[7117, 6, 2]
[22930, 2, 5]
[8, 20, 4]
[8, 3, 7]
[7119, 11, 6]
[5, 6, 9]
[13, 24, 8]
[42, 17, 7127]
[13, 7, 12]
[7130, 15, 11]
[9, 11, 14]
[18, 30, 13]
[16, 18, 12]
[7131, 17, 15]
[10, 43, 16]
[14, 15, 19]
[47, 34, 18]
[21, 5, 25]
[22940, 20, 22]
[23, 21, 27]
[22941, 22, 5385]
[25, 9, 30]
[20, 24, 26]
[28, 27, 25]
[5387, 22, 26]
[32, 29, 26]
[5393, 28, 39]
[24, 14, 31]
[35, 32, 30]
[28, 31, 33]
[37, 38, 32]
[35, 19, 59]
[31, 34, 36]
[37, 35, 60]
[33, 36, 40]
[39, 33, 40]
[29, 38, 5398]
[38, 37, 41]
[67, 5401, 40]
[43, 10, 7135]
[44, 17, 42]
[45, 46, 43]
[7142, 48, 44]
[47, 44, 51]
[19, 46, 58]
[49, 45, 7144]
[50, 48, 52]
0.11658802032470703
